## Sentiment pipepline

Write down your tweet, and roBERTa will found out its sentiment and the words that reflect this sentiment.

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from sklearn.model_selection import StratifiedKFold
from transformers import *
import tokenizers
import os

In [2]:
path = os.getcwd()
max_length = 96

### Loading Tokenizer

In [3]:
tokenizer = tokenizers.ByteLevelBPETokenizer(
    vocab= path + '/vocab.json',
    merges = path + '/merges.txt',
    lowercase = True, #All tokens are in lower case
    add_prefix_space=True #Do not treat spaces like part of the tokens
)

sentiment_id = {'positive': [1, 0, 0], 'negative': [0, 0, 1], 'neutral': [0, 1, 0]} 
sentiment_rev = ['positive', 'neutral', 'negative']
sentiment_token_id = {'positive': 1313, 'negative': 2430, 'neutral': 7974} 

train_set = pd.read_csv(path+'/train.csv').fillna('')
train_set.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


### Creating models' architectures

In [4]:
def classify_model():
    ids = tf.keras.layers.Input((max_length,), dtype=tf.int32)
    att = tf.keras.layers.Input((max_length,), dtype=tf.int32)
    tok = tf.keras.layers.Input((max_length,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(path+'/config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(path+'/pretrained-roberta-base.h5',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
        
    x3 = tf.keras.layers.Dropout(0.1)(x[0])
    x3 = tf.keras.layers.Conv1D(8, 2, padding='same')(x3) #3
    x3 = tf.keras.layers.LeakyReLU()(x3)
    x3 = tf.keras.layers.Conv1D(4, 2, padding='same')(x3) #3
    x3 = tf.keras.layers.Flatten()(x3)
    #x3 = tf.keras.layers.Dense(16, activation='relu')(x3) #uncommented
    x3 = tf.keras.layers.Dense(8, activation='relu')(x3)
    x3 = tf.keras.layers.Dense(3, activation='relu')(x3)
    x3 = tf.keras.layers.Activation('softmax')(x3)
    

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x3])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

In [5]:
def extract_model():
    ids = tf.keras.layers.Input((max_length,), dtype=tf.int32)
    att = tf.keras.layers.Input((max_length,), dtype=tf.int32)
    tok = tf.keras.layers.Input((max_length,), dtype=tf.int32)

    config = RobertaConfig.from_pretrained(path+'/config-roberta-base.json')
    bert_model = TFRobertaModel.from_pretrained(path+'/pretrained-roberta-base.h5',config=config)
    x = bert_model(ids,attention_mask=att,token_type_ids=tok)
    
    x1 = tf.keras.layers.Dropout(0.1)(x[0])
    x1 = tf.keras.layers.Conv1D(filters=128, kernel_size=2, padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(filters=64, kernel_size=2, padding='same')(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Conv1D(filters=32, kernel_size=2, padding='same')(x1)
    x1 = tf.keras.layers.Dense(units=1)(x1)
    x1 = tf.keras.layers.LeakyReLU()(x1)
    x1 = tf.keras.layers.Flatten()(x1)
    x1 = tf.keras.layers.Activation('softmax')(x1)
    
    x2 = tf.keras.layers.Dropout(0.1)(x[0])
    x2 = tf.keras.layers.Conv1D(filters=128, kernel_size=2, padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(filters=64, kernel_size=2, padding='same')(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Conv1D(filters=32, kernel_size=2, padding='same')(x2)
    x2 = tf.keras.layers.Dense(units=1)(x2)
    x2 = tf.keras.layers.LeakyReLU()(x2)
    x2 = tf.keras.layers.Flatten()(x2)
    x2 = tf.keras.layers.Activation('softmax')(x2)

    model = tf.keras.models.Model(inputs=[ids, att, tok], outputs=[x1,x2])
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    model.compile(loss='categorical_crossentropy', optimizer=optimizer)

    return model

### Loading trained models

In [6]:
extract_m = extract_model()
extract_m.load_weights(path+'/v0-roberta-4-extract.h5')

classify_m = classify_model()
classify_m.load_weights(path+'/v1-roberta-ext-1-classify.h5')

All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at /home/lucas/Documents/PO-240/sentiment-extraction/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
All model checkpoint layers were used when initializing TFRobertaModel.

All the layers of TFRobertaModel were initialized from the model checkpoint at /home/lucas/Documents/PO-240/sentiment-extraction/pretrained-roberta-base.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


### Predict tweet sentiment

Type your tweet below and roBERTa will predict its sentiment.

In [26]:
input_ids = np.ones((1, max_length), dtype='int32')
attention_mask = np.zeros((1, max_length), dtype='int32')
token_type_ids = np.zeros((1, max_length),dtype='int32')


#Type your tweet here
text = 'Ford Brasi\'s decision to close its factories is partly a result of Ford\'s global woes. It also shows the weakness of Brazilian manufacturing'
if len(text) > 140:
    raise Exception('Text too big')

In [27]:
text1 = " " + " ".join(text.split())
enc = tokenizer.encode(text1)
input_ids[0, :len(enc.ids)+2] = [0] + enc.ids + [2]
attention_mask[0, :len(enc.ids)+2] = 1

result = classify_m.predict([input_ids, attention_mask, token_type_ids], verbose=True)
s_id = np.argmax(result[0])
print()
print('Sentiment: ', sentiment_rev[s_id])

1/1 [==============================] - 0s 958us/step

Sentiment:  negative


In [28]:
sentiment_token = sentiment_token_id[sentiment_rev[s_id]]
input_ids[0, :len(enc.ids)+5] = [0] + enc.ids + [2,2] + [sentiment_token] + [2]
attention_mask[0, :len(enc.ids)+5] = 1

start, end = extract_m.predict([input_ids, attention_mask, token_type_ids], verbose=True)
start_idx = np.argmax(start)
end_idx = np.argmax(end)
selected_text = tokenizer.decode(enc.ids[start_idx-1:end_idx])
print('\nSelected text:')
print(selected_text)



1/1 [==============================] - 0s 2ms/step

Selected text:
 woes. it also shows the weakness of brazilian manufacturing
